# First Attempt with Resnet

- Author : Areeb Islam
- Github : are021

In [ ]:
! pip install --upgrade torch torchvision

In [ ]:
import torch
from torchvision import transforms, datasets
from PIL import Image

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()


In [ ]:
# Freeze layers
for param in model.parameters():
  param.requires_grad = False

In [ ]:
num_features = model.fc.in_features

# Change last layer to binary (recyclable or non-recyclable)
# We may need to change this architecture to first classify is recyclable, then contaminated
model.fc = torch.nn.Linear(num_features, 2)

In [ ]:
model.eval()

In [ ]:
# Define the image transform
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load our dataset
I am using this dataset I found on kaggle, I want to just give it a try and test it's validity.

In [ ]:
! mkdir .kaggle

In [ ]:
! chmod 600 .kaggle/kaggle.json


In [ ]:
! kaggle datasets download techsash/waste-classification-data/

In [ ]:
!unzip waste-classification-data.zip

In [ ]:
test_path = '/content/dataset/DATASET/TEST'
train_path = '/content/dataset/DATASET/TRAIN'

In [ ]:
training_data = datasets.ImageFolder(train_path, transform=preprocess)
training_data

In [ ]:
testing_data = datasets.ImageFolder(test_path, transform=preprocess)
testing_data

In [ ]:
first_image = testing_data[0][0]
first_image.size()

In [ ]:
# Lets visualize the image
import matplotlib.pyplot as plt
import numpy as np

first_image_np = first_image.numpy()

plt.imshow(first_image_np.T, cmap='gray')
plt.show()

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# Define the dataloader
batch_size = 32

training_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
testing_dataloader =  DataLoader(testing_data, batch_size=batch_size, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# define number of epochs
num_epochs = 10

In [ ]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

for epoch in range(num_epochs):
    running_loss = 0.0
    model.train(True)
    for inputs, labels in tqdm(training_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch+1} loss: {running_loss / len(training_data)}')

In [ ]:
path = './model'
torch.save(model.state_dict(), path)

In [ ]:
def calculate_accuracy(model, dataloader, device):
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += torch.sum(predicted == labels).item()
    accuracy = 100 * correct / total
    return accuracy

test_accuracy = calculate_accuracy(model, testing_dataloader, device)
print(f'\nTest accuracy: {test_accuracy:.2f}%')